In [2]:
## Here we want to use KNN algorithm to generate the mostly like samples 
## At first, we only use the meta information to do the KNN 
## load data
import json
import os
import numpy as np
import pandas as pd
# import lightgbm as lgbm
from datetime import datetime
from time import gmtime, strftime
from scipy import stats
# from PIL import Image
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.model_selection import StratifiedKFold, KFold
from sklearn.metrics import f1_score, mean_absolute_error, mean_squared_error
from scipy import stats
from scipy.stats import norm, skew #for some statistics

In [ ]:
## ['Uid', 'Category', 'Subcategory', 'Concept', 'Mediatype',] --> dense_features1
## ['photo_count', 'totalTags', 'totalGeotagged', 'totalFaves','totalInGroup','photoCount','meanView', 'meanTags', 'meanFaves', 'followerCount','followingCount']
## --> dense_features2
## ['hour', 'day', 'weekday', 'week_hour',]  --> time_series


In [5]:
all_data = pd.read_csv('../data/feature_data_530.csv')
## 决定使用爬取的meta信息作为KNN聚类特征，首先选择这部分特征，接着将这些特征取log
fea = ['Uid', 'train_type','Title_number', 'Alltags_number', 'totalTags','totalFaves','totalInGroup','meanView', 'followerCount','followingCount','label']
all_data = all_data[fea]



In [6]:
all_data

,Uid,train_type,Title_number,Alltags_number,totalTags,totalFaves,totalInGroup,meanView,followerCount,followingCount,label
0,21894,0,6,12,2215,904,176,4299.470263,2405,2729,11.18
1,53866,0,13,65,3991,6200,679,735.884072,8491,236,15.15
2,26948,0,2,23,2930,2583,2,5124.185756,1550,1346,10.99
3,355,0,3,9,5707,76088,385,148.035342,1233,1115,8.63
4,315,0,1,19,5344,0,102,5300.882729,5781,13,11.16
...,...,...,...,...,...,...,...,...,...,...,...
486189,46088,-1,1,13,987,4341,84,833.933718,489,612,0.00
486190,12280,-1,4,23,0,0,0,0.000000,0,0,0.00
486191,43496,-1,7,9,21861,5225,98,1611.040268,702,113,0.00
486192,5492,-1,3,35,5677,24679,373,2480.537418,1092,389,0.00


In [7]:
train_all_data = all_data[all_data['train_type'] != -1]
submit_all_data = all_data[all_data['train_type'] == -1]
train_all_data = train_all_data.reset_index(drop=True)
submit_all_data = submit_all_data.reset_index(drop=True)

In [13]:
feature_columns = ['train_type', 'label']
train_label_df = train_all_data[['label']]
train_feature_df = train_all_data.drop(feature_columns, axis=1)

submit_label_df = submit_all_data[['label']]
submit_feature_df = submit_all_data.drop(feature_columns, axis=1)

In [14]:
columns = ['Title_number', 'Alltags_number', 'totalTags','totalFaves','totalInGroup','meanView', 'followerCount','followingCount']
for index in columns:
    train_feature_df[index] = np.log1p(train_feature_df[index])
    submit_feature_df[index] = np.log1p(submit_feature_df[index])

In [20]:
train_feature_df = pd.concat([train_feature_df, train_label_df], axis=1)
submit_feature_df = pd.concat([submit_feature_df, submit_label_df], axis=1)

In [38]:
all_data_fea = pd.concat([train_feature_df, submit_feature_df], axis=0)
all_data_fea

,Uid,Title_number,Alltags_number,totalTags,totalFaves,totalInGroup,meanView,followerCount,followingCount,label
0,21894,1.945910,2.564949,7.703459,6.807935,5.176150,8.366480,7.785721,7.912057,11.18
1,53866,2.639057,4.189655,8.292048,8.732466,6.522093,6.602431,9.046880,5.468060,15.15
2,26948,1.098612,3.178054,7.983099,7.857094,1.098612,8.541922,7.346655,7.205635,10.99
3,355,1.386294,2.302585,8.649624,11.239659,5.955837,5.004183,7.118016,7.017506,8.63
4,315,0.693147,2.995732,8.583917,0.000000,4.634729,8.575817,8.662505,2.639057,11.16
...,...,...,...,...,...,...,...,...,...,...
180576,46088,0.693147,2.639057,6.895683,8.376090,4.442651,6.727352,6.194405,6.418365,0.00
180577,12280,1.609438,3.178054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00
180578,43496,2.079442,2.302585,9.992505,8.561401,4.595120,7.385256,6.555357,4.736198,0.00
180579,5492,1.386294,3.583519,8.644354,10.113748,5.924256,7.816634,6.996681,5.966147,0.00


In [46]:
all_data_fea = all_data_fea.drop(['Uid', 'label'], axis=1)
# all_data_fea = all_data_fea.reset_index(drop=True)
# submit_all_data.drop(feature_columns, axis=1)

In [67]:
all_data_fea = all_data_fea.reset_index(drop=True)
all_data_fea

,Title_number,Alltags_number,totalTags,totalFaves,totalInGroup,meanView,followerCount,followingCount
0,1.945910,2.564949,7.703459,6.807935,5.176150,8.366480,7.785721,7.912057
1,2.639057,4.189655,8.292048,8.732466,6.522093,6.602431,9.046880,5.468060
2,1.098612,3.178054,7.983099,7.857094,1.098612,8.541922,7.346655,7.205635
3,1.386294,2.302585,8.649624,11.239659,5.955837,5.004183,7.118016,7.017506
4,0.693147,2.995732,8.583917,0.000000,4.634729,8.575817,8.662505,2.639057
...,...,...,...,...,...,...,...,...
486189,0.693147,2.639057,6.895683,8.376090,4.442651,6.727352,6.194405,6.418365
486190,1.609438,3.178054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
486191,2.079442,2.302585,9.992505,8.561401,4.595120,7.385256,6.555357,4.736198
486192,1.386294,3.583519,8.644354,10.113748,5.924256,7.816634,6.996681,5.966147


In [29]:
train_feature_df['label'].max()

16.56

In [68]:
from sklearn.neighbors import NearestNeighbors
# target_feature = 'label'
n_max = 10

# make neighbors
pivot = all_data_fea
# pivot = pivot.fillna(pivot.mean())
# pivot = pd.DataFrame(minmax_scale(pivot))
# pivot = df.pivot('time_id', 'stock_id', 'price')
nn = NearestNeighbors(n_neighbors=n_max, p=1)
nn.fit(pivot)
neighbors = nn.kneighbors(pivot)

# aggregate

# def make_nn_feature(neighbors, f_col, n=5, agg=np.mean, postfix=''):
#     pivot_aggs = pd.DataFrame(agg(neighbors[1:n,:,:], axis=0), 
#                               columns=feature_pivot.columns, 
#                               index=feature_pivot.index)
#     dst = pivot_aggs.unstack().reset_index()
#     dst.columns = dst.columns + [f'{f_col}_cluster{n}{postfix}_{agg.__name__}']
#     return dst

# # feature_pivot = df.pivot('time_id', 'stock_id', target_feature)
# # feature_pivot = feature_pivot.fillna(feature_pivot.mean())
# feature_pivot = pd.concat([pivot, train_label_df], axis=1)
# neighbor_features = np.zeros((n_max, *feature_pivot.shape))

# for i in range(n_max):
#     neighbor_features[i, :, :] += feature_pivot.values[neighbors[:, i], :]

# for n in [2, 3, 5, 10, 20, 40]:
#     dst = make_nn_feature(neighbors, feature_pivot, n)
#     # df = pd.merge(df, dst, on=['stock_id', 'time_id'], how='left')
# dst

In [69]:
distances, indices = neighbors
pd.DataFrame(distances)

,0,1,2,3,4,5,6,7,8,9
0,0.0,0.154151,0.154151,1.054937,2.201298,2.351565,2.485033,2.535899,2.537771,2.543862
1,0.0,0.184004,0.493658,0.595086,0.675129,0.767255,1.124930,1.130424,1.193922,1.368276
2,0.0,3.291280,3.325182,3.357972,3.389721,3.416164,3.767848,3.832386,3.832386,4.055530
3,0.0,1.918540,1.953967,1.953967,1.953967,1.953967,1.953967,2.246449,2.272420,2.277662
4,0.0,0.223144,0.223144,0.223144,0.262364,0.262364,0.300105,0.300105,0.336472,0.336472
...,...,...,...,...,...,...,...,...,...,...
486189,0.0,0.944462,1.020060,1.020060,1.288150,1.314212,1.421318,1.483058,1.499633,1.559057
486190,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
486191,0.0,0.587787,0.652325,0.693147,0.818310,0.818310,0.832909,0.929536,1.163151,2.302319
486192,0.0,0.054067,0.105361,0.149532,0.287682,0.315853,0.340927,0.344840,0.405465,0.432864


In [70]:
pd.DataFrame(indices)

,0,1,2,3,4,5,6,7,8,9
0,0,139,307385,218646,143665,114838,284051,294451,174586,373189
1,1,128,144,317186,130,317185,141,143,125,317187
2,2,264769,260243,262651,263999,239327,325235,325373,322297,121671
3,3,414405,303952,296601,304789,296602,290117,224557,206015,253145
4,4,123145,149826,131513,126120,120562,121707,131840,122107,132050
...,...,...,...,...,...,...,...,...,...,...
486189,486189,203645,268491,327432,279151,446983,254365,355151,327614,268740
486190,5062,23263,9638,12591,7640,5061,9134,7657,7390,7537
486191,486191,196455,470924,364194,423167,373521,130966,120908,203970,234201
486192,486192,363853,473960,29080,434992,227375,146224,474194,171283,238007


In [58]:
indices[1]

array([     1,    128,    144, 317186,    130, 317185,    141,    143,
          125, 317187])

In [59]:
## 无监督的KNN看起来效果似乎还行，接下来要做的就是拼接，挨个将对应的列拼接起来，并放入对应的列中
## 先试着提取第1，2列的作为特征，再提取1-6列的，1-9列的均值作为特征
indices_df = pd.DataFrame(indices)
np.array(indices_df[1])



#最后将原来的UID拼接上 

array([   139,    128, 264769, ..., 196455, 363853, 417922])

In [71]:
knn1 = all_data_fea.iloc[np.array(indices_df[1])]
knn1.columns = ['knn1_Title_number', 'knn1_Alltags_number', 'knn1_totalTags','knn1_totalFaves','knn1_totalInGroup','knn1_meanView', 'knn1_followerCount','knn1_followingCount']
knn1 = knn1.reset_index(drop=True)
knn1

,knn1_Title_number,knn1_Alltags_number,knn1_totalTags,knn1_totalFaves,knn1_totalInGroup,knn1_meanView,knn1_followerCount,knn1_followingCount
0,1.791759,2.564949,7.703459,6.807935,5.176150,8.366480,7.785721,7.912057
1,2.484907,4.219508,8.292048,8.732466,6.522093,6.602431,9.046880,5.468060
2,1.098612,3.367296,8.434246,8.893298,0.693147,8.349716,8.045588,6.887553
3,1.386294,2.302585,8.886409,10.535371,5.991465,5.617346,6.862758,6.944087
4,0.693147,3.218876,8.583917,0.000000,4.634729,8.575817,8.662505,2.639057
...,...,...,...,...,...,...,...,...
486189,1.098612,3.178054,6.895683,8.376090,4.442651,6.727352,6.194405,6.418365
486190,1.609438,3.178054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
486191,2.079442,2.890372,9.992505,8.561401,4.595120,7.385256,6.555357,4.736198
486192,1.386294,3.637586,8.644354,10.113748,5.924256,7.816634,6.996681,5.966147


In [72]:
knn2 = all_data_fea.iloc[np.array(indices_df[2])]
knn2.columns = ['knn2_Title_number', 'knn2_Alltags_number', 'knn2_totalTags','knn2_totalFaves','knn2_totalInGroup','knn2_meanView', 'knn2_followerCount','knn2_followingCount']
knn2 = knn2.reset_index(drop=True)
knn2

,knn2_Title_number,knn2_Alltags_number,knn2_totalTags,knn2_totalFaves,knn2_totalInGroup,knn2_meanView,knn2_followerCount,knn2_followingCount
0,1.791759,2.564949,7.703459,6.807935,5.176150,8.366480,7.785721,7.912057
1,2.484907,3.850148,8.292048,8.732466,6.522093,6.602431,9.046880,5.468060
2,1.098612,3.401197,8.434246,8.893298,0.693147,8.349716,8.045588,6.887553
3,1.386294,2.302585,8.509161,11.226096,6.001415,6.316100,6.792344,6.900731
4,0.693147,3.218876,8.583917,0.000000,4.634729,8.575817,8.662505,2.639057
...,...,...,...,...,...,...,...,...
486189,0.693147,2.639057,6.874198,8.462103,4.795791,6.423525,6.359574,6.327937
486190,1.609438,3.178054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
486191,1.609438,2.484907,9.992505,8.561401,4.595120,7.385256,6.555357,4.736198
486192,1.386294,3.688879,8.644354,10.113748,5.924256,7.816634,6.996681,5.966147


In [74]:
k1 = all_data_fea.iloc[np.array(indices_df[1])]
k1 = k1.reset_index(drop=True)
k2 = all_data_fea.iloc[np.array(indices_df[2])]
k2 = k2.reset_index(drop=True)
k3 = all_data_fea.iloc[np.array(indices_df[3])]
k3 = k3.reset_index(drop=True)
k4 = all_data_fea.iloc[np.array(indices_df[4])]
k4 = k4.reset_index(drop=True)
k5 = all_data_fea.iloc[np.array(indices_df[5])]
k5 = k5.reset_index(drop=True)
k6 = all_data_fea.iloc[np.array(indices_df[6])]
k6 = k6.reset_index(drop=True)
k7 = all_data_fea.iloc[np.array(indices_df[7])]
k7 = k7.reset_index(drop=True)
k8 = all_data_fea.iloc[np.array(indices_df[8])]
k8 = k8.reset_index(drop=True)
k9 = all_data_fea.iloc[np.array(indices_df[9])]
k9 = k9.reset_index(drop=True)

,Title_number,Alltags_number,totalTags,totalFaves,totalInGroup,meanView,followerCount,followingCount
0,4.836282,7.976595,22.541809,21.240000,15.620158,25.385252,23.227779,23.971692
1,7.901007,11.466588,24.876143,26.197398,19.566278,19.807292,27.140639,16.404180
2,3.295837,10.267019,25.302739,26.679894,2.079442,25.049148,24.136765,20.662658
3,4.158883,6.907755,25.904731,32.987564,17.994294,18.249545,20.447447,20.745549
4,2.079442,9.695848,25.751750,0.000000,13.904187,25.727452,25.987515,7.917172
...,...,...,...,...,...,...,...,...
486189,2.484907,8.456168,20.624236,25.912943,13.632891,20.312836,18.792304,19.143231
486190,4.828314,9.534161,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
486191,5.662960,8.431635,29.977516,25.684204,13.785360,22.155768,19.666071,14.208595
486192,3.871201,10.655092,25.933063,30.341245,17.772767,23.449901,20.990044,17.898440


In [81]:
mean5 = (k1 + k2 + k3 + k4 + k5)/5
mean5

,Title_number,Alltags_number,totalTags,totalFaves,totalInGroup,meanView,followerCount,followingCount
0,1.647496,2.621299,7.550152,6.910744,5.226769,8.457747,7.825625,7.788329
1,2.556762,3.796060,8.292048,8.732466,6.522093,6.602431,9.046880,5.468060
2,1.156149,3.261455,8.322952,8.738340,0.912870,8.233958,8.037344,6.971348
3,1.386294,2.302585,8.584611,11.087951,5.999425,6.176349,6.806427,6.909402
4,0.693147,3.234564,8.583917,0.000000,4.634729,8.575817,8.662505,2.639057
...,...,...,...,...,...,...,...,...
486189,0.774240,2.658490,6.831546,8.562942,4.562142,6.755275,6.279984,6.407005
486190,1.609438,3.178054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
486191,1.812832,2.749951,9.992505,8.561401,4.595120,7.385256,6.555357,4.736198
486192,1.271222,3.579864,8.644354,10.113748,5.924256,7.816634,6.996681,5.966147


In [82]:
mean9 = (k1 + k2 + k3 + k4 + k5 + k6 + k7 + k8 + k9)/9
mean9

,Title_number,Alltags_number,totalTags,totalFaves,totalInGroup,meanView,followerCount,followingCount
0,1.678268,2.513879,7.594559,6.845014,5.235832,8.283533,7.945490,7.948615
1,2.355071,3.711706,8.292048,8.732466,6.522093,6.602431,9.046880,5.468060
2,1.085525,3.042926,7.724876,7.870952,1.123281,8.305524,8.097766,7.086555
3,1.456140,2.346520,8.512279,11.086564,5.932226,6.048205,6.807245,6.890972
4,0.693147,3.269878,8.583917,0.000000,4.634729,8.575817,8.662505,2.639057
...,...,...,...,...,...,...,...,...
486189,0.828302,2.664122,6.862299,8.530432,4.620584,6.627369,6.296244,6.385491
486190,1.609438,3.178054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
486191,1.847138,2.767207,9.866774,8.534195,4.609816,7.372614,6.525841,4.757427
486192,1.328281,3.552008,8.644354,10.113748,5.924256,7.816634,6.996681,5.966147


In [83]:
mean5.columns = ['mean5_Title_number', 'mean5_Alltags_number', 'mean5_totalTags','mean5_totalFaves','mean5_totalInGroup','mean5_meanView', 'mean5_followerCount','mean5_followingCount']
mean9.columns = ['mean9_Title_number', 'mean9_Alltags_number', 'mean9_totalTags','mean9_totalFaves','mean9_totalInGroup','mean9_meanView', 'mean9_followerCount','mean9_followingCount']

In [87]:
re_all_data_fea.columns

Index(['Title_number', 'Alltags_number', 'totalTags', 'totalFaves',
       'totalInGroup', 'meanView', 'followerCount', 'followingCount',
       'knn1_Title_number', 'knn1_Alltags_number', 'knn1_totalTags',
       'knn1_totalFaves', 'knn1_totalInGroup', 'knn1_meanView',
       'knn1_followerCount', 'knn1_followingCount', 'knn2_Title_number',
       'knn2_Alltags_number', 'knn2_totalTags', 'knn2_totalFaves',
       'knn2_totalInGroup', 'knn2_meanView', 'knn2_followerCount',
       'knn2_followingCount', 'mean5_Title_number', 'mean5_Alltags_number',
       'mean5_totalTags', 'mean5_totalFaves', 'mean5_totalInGroup',
       'mean5_meanView', 'mean5_followerCount', 'mean5_followingCount',
       'mean9_Title_number', 'mean9_Alltags_number', 'mean9_totalTags',
       'mean9_totalFaves', 'mean9_totalInGroup', 'mean9_meanView',
       'mean9_followerCount', 'mean9_followingCount'],
      dtype='object')

In [73]:
re_all_data_fea = pd.concat([all_data_fea, knn1, knn2], axis=1)
re_all_data_fea

,Title_number,Alltags_number,totalTags,totalFaves,totalInGroup,meanView,followerCount,followingCount,knn1_Title_number,knn1_Alltags_number,...,knn1_followerCount,knn1_followingCount,knn2_Title_number,knn2_Alltags_number,knn2_totalTags,knn2_totalFaves,knn2_totalInGroup,knn2_meanView,knn2_followerCount,knn2_followingCount
0,1.945910,2.564949,7.703459,6.807935,5.176150,8.366480,7.785721,7.912057,1.791759,2.564949,...,7.785721,7.912057,1.791759,2.564949,7.703459,6.807935,5.176150,8.366480,7.785721,7.912057
1,2.639057,4.189655,8.292048,8.732466,6.522093,6.602431,9.046880,5.468060,2.484907,4.219508,...,9.046880,5.468060,2.484907,3.850148,8.292048,8.732466,6.522093,6.602431,9.046880,5.468060
2,1.098612,3.178054,7.983099,7.857094,1.098612,8.541922,7.346655,7.205635,1.098612,3.367296,...,8.045588,6.887553,1.098612,3.401197,8.434246,8.893298,0.693147,8.349716,8.045588,6.887553
3,1.386294,2.302585,8.649624,11.239659,5.955837,5.004183,7.118016,7.017506,1.386294,2.302585,...,6.862758,6.944087,1.386294,2.302585,8.509161,11.226096,6.001415,6.316100,6.792344,6.900731
4,0.693147,2.995732,8.583917,0.000000,4.634729,8.575817,8.662505,2.639057,0.693147,3.218876,...,8.662505,2.639057,0.693147,3.218876,8.583917,0.000000,4.634729,8.575817,8.662505,2.639057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,0.693147,2.639057,6.895683,8.376090,4.442651,6.727352,6.194405,6.418365,1.098612,3.178054,...,6.194405,6.418365,0.693147,2.639057,6.874198,8.462103,4.795791,6.423525,6.359574,6.327937
486190,1.609438,3.178054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.609438,3.178054,...,0.000000,0.000000,1.609438,3.178054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
486191,2.079442,2.302585,9.992505,8.561401,4.595120,7.385256,6.555357,4.736198,2.079442,2.890372,...,6.555357,4.736198,1.609438,2.484907,9.992505,8.561401,4.595120,7.385256,6.555357,4.736198
486192,1.386294,3.583519,8.644354,10.113748,5.924256,7.816634,6.996681,5.966147,1.386294,3.637586,...,6.996681,5.966147,1.386294,3.688879,8.644354,10.113748,5.924256,7.816634,6.996681,5.966147


In [84]:
re_all_data_fea = pd.concat([re_all_data_fea, mean5, mean9], axis=1)
re_all_data_fea

,Title_number,Alltags_number,totalTags,totalFaves,totalInGroup,meanView,followerCount,followingCount,knn1_Title_number,knn1_Alltags_number,...,mean5_followerCount,mean5_followingCount,mean9_Title_number,mean9_Alltags_number,mean9_totalTags,mean9_totalFaves,mean9_totalInGroup,mean9_meanView,mean9_followerCount,mean9_followingCount
0,1.945910,2.564949,7.703459,6.807935,5.176150,8.366480,7.785721,7.912057,1.791759,2.564949,...,7.825625,7.788329,1.678268,2.513879,7.594559,6.845014,5.235832,8.283533,7.945490,7.948615
1,2.639057,4.189655,8.292048,8.732466,6.522093,6.602431,9.046880,5.468060,2.484907,4.219508,...,9.046880,5.468060,2.355071,3.711706,8.292048,8.732466,6.522093,6.602431,9.046880,5.468060
2,1.098612,3.178054,7.983099,7.857094,1.098612,8.541922,7.346655,7.205635,1.098612,3.367296,...,8.037344,6.971348,1.085525,3.042926,7.724876,7.870952,1.123281,8.305524,8.097766,7.086555
3,1.386294,2.302585,8.649624,11.239659,5.955837,5.004183,7.118016,7.017506,1.386294,2.302585,...,6.806427,6.909402,1.456140,2.346520,8.512279,11.086564,5.932226,6.048205,6.807245,6.890972
4,0.693147,2.995732,8.583917,0.000000,4.634729,8.575817,8.662505,2.639057,0.693147,3.218876,...,8.662505,2.639057,0.693147,3.269878,8.583917,0.000000,4.634729,8.575817,8.662505,2.639057
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
486189,0.693147,2.639057,6.895683,8.376090,4.442651,6.727352,6.194405,6.418365,1.098612,3.178054,...,6.279984,6.407005,0.828302,2.664122,6.862299,8.530432,4.620584,6.627369,6.296244,6.385491
486190,1.609438,3.178054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.609438,3.178054,...,0.000000,0.000000,1.609438,3.178054,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
486191,2.079442,2.302585,9.992505,8.561401,4.595120,7.385256,6.555357,4.736198,2.079442,2.890372,...,6.555357,4.736198,1.847138,2.767207,9.866774,8.534195,4.609816,7.372614,6.525841,4.757427
486192,1.386294,3.583519,8.644354,10.113748,5.924256,7.816634,6.996681,5.966147,1.386294,3.637586,...,6.996681,5.966147,1.328281,3.552008,8.644354,10.113748,5.924256,7.816634,6.996681,5.966147


In [86]:
re_all_data_fea.to_csv('./KNN_fea.csv',header=True, index=None)

In [ ]:
knn2 = all_data_fea.iloc[np.array(indices_df[2])]
knn2.columns = ['knn2_Title_number', 'knn2_Alltags_number', 'knn2_totalTags','knn2_totalFaves','knn2_totalInGroup','knn2_meanView', 'knn2_followerCount','knn2_followingCount']
knn2 = knn2.reset_index(drop=True)
knn2

In [33]:
pd.DataFrame(neighbors[0])

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0.0,1.094937,1.334151,8.296990,10.839408,15.459373,16.625019,19.033670,19.610612,19.935522,...,21.964738,21.964738,21.981438,22.153242,22.175560,22.241410,22.305196,25.761432,27.017799,27.081718
1,0.0,0.963658,1.285129,1.504004,2.630424,2.674930,3.963922,19.047255,19.345092,20.085867,...,30.029079,30.054663,30.199079,30.475334,30.762565,30.920799,30.976649,31.036649,31.180247,31.236264
2,0.0,18.520749,18.599438,18.674857,21.274657,21.994443,22.125125,22.161957,22.308084,22.454710,...,22.767646,22.791957,22.834710,22.872339,22.876486,22.877646,22.914657,22.919167,22.966486,22.986874
3,0.0,11.760593,12.138843,12.343170,12.878271,13.111312,13.306314,18.734243,24.850896,28.712157,...,36.651792,36.746550,37.620081,37.714759,38.190818,38.310251,38.412442,39.459155,40.098805,42.848783
4,0.0,0.506472,1.053144,1.125465,1.181854,1.712364,1.772364,1.861564,1.883144,1.926472,...,2.540105,2.832763,3.320466,3.337858,4.026015,5.108148,25.370697,25.664341,26.383945,26.500801
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305608,0.0,3.196780,3.288997,3.386223,3.411314,4.082322,10.093069,10.115910,10.153693,10.160550,...,11.127228,11.193069,11.193069,11.253693,11.260550,11.260550,11.279057,11.282690,11.307228,11.367228
305609,0.0,14.489801,16.516276,18.051071,18.051071,18.161071,18.351071,18.351071,18.441071,18.441071,...,31.622596,31.622596,31.632596,31.642596,31.642596,31.642596,31.642596,31.642596,31.642596,31.642596
305610,0.0,5.003248,15.406285,18.298378,18.725955,18.817734,20.092424,21.957307,22.186934,22.417695,...,31.757727,32.692143,32.779403,33.053070,33.110752,33.210752,33.250752,33.300752,33.340752,33.403087
305611,0.0,1.218790,1.807298,2.146088,2.332519,2.619762,2.714437,2.837682,3.070105,3.100826,...,3.825310,3.866472,3.871937,3.878255,3.940826,3.982519,4.012992,4.055508,4.067787,4.083147


In [32]:
pd.DataFrame(neighbors[1])

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,0,218646,139,15592,247925,95626,118799,289345,22883,146587,...,94337,76989,94019,89205,95636,94336,228549,59711,180842,212445
1,1,144,130,128,143,141,125,44455,121557,121416,...,270086,185465,262644,170752,179066,185464,155516,155412,147864,170753
2,2,172344,175603,151093,119934,283809,135521,46116,177834,120891,...,130595,48430,142510,120889,144007,144010,119930,46438,134071,130596
3,3,204873,74207,181125,64589,230156,177762,301598,224612,20910,...,202636,236730,250812,49788,62241,209905,44469,295117,100262,3530
4,4,149856,149826,135740,7,126120,120562,138636,123145,122107,...,121707,132207,5,132513,136883,8,33564,43935,13270,4859
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305608,305608,256405,279461,303813,296188,251336,295329,293520,295334,289300,...,268198,295324,295326,268194,293957,293478,293966,293978,289301,293968
305609,305609,225149,105129,279568,300759,297609,297611,297610,298618,297612,...,92568,112888,112628,99586,112195,113560,90724,97877,113233,112194
305610,305610,286892,35559,193280,187869,222771,164386,82144,287255,82145,...,281859,6839,117564,255105,255108,280145,255109,287455,280146,108426
305611,305611,300095,290246,292141,304945,299150,291953,285215,291168,297569,...,298743,281220,298097,299541,297823,286288,274523,254512,258001,296522


In [88]:
!pip install gokinjo

In [91]:
train_feature_df = train_feature_df.drop(['label'], axis=1)
train_feature_df

,Uid,Title_number,Alltags_number,totalTags,totalFaves,totalInGroup,meanView,followerCount,followingCount
0,21894,1.945910,2.564949,7.703459,6.807935,5.176150,8.366480,7.785721,7.912057
1,53866,2.639057,4.189655,8.292048,8.732466,6.522093,6.602431,9.046880,5.468060
2,26948,1.098612,3.178054,7.983099,7.857094,1.098612,8.541922,7.346655,7.205635
3,355,1.386294,2.302585,8.649624,11.239659,5.955837,5.004183,7.118016,7.017506
4,315,0.693147,2.995732,8.583917,0.000000,4.634729,8.575817,8.662505,2.639057
...,...,...,...,...,...,...,...,...,...
305608,8689,0.693147,3.583519,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
305609,7509,0.693147,2.302585,6.489205,6.021023,3.496508,6.990934,4.812184,3.688879
305610,52220,0.693147,2.197225,6.892642,8.505121,5.313206,7.201785,7.118826,6.896694
305611,20298,1.386294,2.995732,9.989665,14.071193,7.966587,8.243203,9.651816,9.849348


In [90]:
train_label_df

,label
0,11.18
1,15.15
2,10.99
3,8.63
4,11.16
...,...
305608,4.00
305609,1.00
305610,2.00
305611,6.89


In [94]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(train_label_df['label'])

In [96]:
# y.max()

1064

In [107]:
from gokinjo import knn_kfold_extract
from gokinjo import knn_extract

X = train_feature_df
# y = train_label_df
X = X.to_numpy()
# y = y.to_numpy()
KNN_feat_train = knn_kfold_extract(X, y, k=1, normalize='standard')
print("KNN features for training set, shape: ", np.shape(KNN_feat_train))
KNN_feat_train[0]

/usr/local/lib/python3.8/dist-packages/sklearn/model_selection/_split.py:700: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  warnings.warn(


ValueError: all the input array dimensions except for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 1066 and the array at index 1 has size 1055

In [106]:
X.shape

(305613, 9)

In [100]:
train_feature_df = train_feature_df.drop(['Uid'], axis=1)
train_feature_df

,Title_number,Alltags_number,totalTags,totalFaves,totalInGroup,meanView,followerCount,followingCount
0,1.945910,2.564949,7.703459,6.807935,5.176150,8.366480,7.785721,7.912057
1,2.639057,4.189655,8.292048,8.732466,6.522093,6.602431,9.046880,5.468060
2,1.098612,3.178054,7.983099,7.857094,1.098612,8.541922,7.346655,7.205635
3,1.386294,2.302585,8.649624,11.239659,5.955837,5.004183,7.118016,7.017506
4,0.693147,2.995732,8.583917,0.000000,4.634729,8.575817,8.662505,2.639057
...,...,...,...,...,...,...,...,...
305608,0.693147,3.583519,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
305609,0.693147,2.302585,6.489205,6.021023,3.496508,6.990934,4.812184,3.688879
305610,0.693147,2.197225,6.892642,8.505121,5.313206,7.201785,7.118826,6.896694
305611,1.386294,2.995732,9.989665,14.071193,7.966587,8.243203,9.651816,9.849348


In [102]:
len(y)

305613

In [ ]:
submit_feature_df

In [ ]:
submit_feature_df

In [ ]:

KNN_feat_test = knn_extract(X, y, X_test, k=1, normalize='standard')
print("KNN features for test set, shape: ", np.shape(KNN_feat_test))
KNN_feat_test[0]